In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, interpolate

import utils
import signalgo

plt.rcParams['figure.figsize'] = [10, 2.5]

In [ ]:
fs = 256  # sampling frequency
df = utils.get_ppgbcg('./sample/ppgbcg.csv')

In [ ]:
# Raw Data
sig0 = -df['CH3'].to_numpy()
plt.plot(sig0)

In [ ]:
# Mean Subtraction
sig1 = sig0 - sig0.mean()
t = np.arange(len(sig1)) / fs
plt.plot(t, sig1, label='sig1')
plt.ylim([-1500, 1500])
plt.xlabel('Time (s)')
plt.legend()

In [ ]:
# Median Filtering
sig2 = signal.medfilt(sig1, kernel_size=3)
t = np.arange(len(sig2)) / fs
plt.plot(t, sig2, label='sig2')
plt.ylim([-1500, 1500])
plt.xlabel('Time (s)')
plt.legend()

In [ ]:
# Band Pass Filtering
filt = signal.firwin(numtaps=1024, cutoff=[1, 5], pass_zero='bandpass', fs=256)
sig3 = signal.convolve(sig2, filt, mode='same')
t = np.arange(len(sig3)) / fs
plt.plot(t, sig2, label='sig2')
plt.plot(t, sig3, label='sig3')
plt.ylim([-1000, 1000])
plt.xlabel('Time (s)')
plt.legend()

In [ ]:
# FFT Results
freq, value = signalgo.fft(sig2, fs, [0.1, 10])
plt.plot(freq, value, label='sig2')
plt.xlabel('Frequency (Hz)')

freq, value = signalgo.fft(sig3, fs, [0.1, 10])
plt.plot(freq, value, label='sig3')
plt.xlabel('Frequency (Hz)')

plt.legend()

In [ ]:
# BPF Visualization
filt_pad = np.concatenate([np.zeros(10000), filt, np.zeros(10000)])
freq, value = signalgo.fft(filt_pad, fs, [0, 10])
t = (np.arange(len(filt)) - len(filt) / 2) / fs
plt.subplot(1, 2, 1)
plt.plot(t, filt, label='filt')
plt.xlabel('Time (s)')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(freq, value, label='filt')
plt.xlabel('Frequency (Hz)')
plt.legend()

In [ ]:
# Signal Normalization
sig4 = signalgo.normalize(sig3)

t = np.arange(len(sig4)) / fs
plt.plot(t, sig4, label='sig3')
plt.xlabel('Time (s)')
plt.legend()

In [ ]:
# Find Peaks
peaks, _ = signal.find_peaks(sig4, prominence=1)
t = np.arange(len(sig4)) / fs
plt.plot(t, sig4, label='sig4')
plt.plot(t[peaks], sig4[peaks], 'o', label='peaks')
plt.ylim([-3, 3])
plt.xlabel('Time (s)')
plt.legend()

In [ ]:
freq, value = signalgo.fft(sig4, sampling_frequency=fs, frequency_range=[0, 2.5])
freq = freq * 60
value = value / len(sig4)
i_max = np.argmax(value)
f_max = freq[i_max]
plt.plot(freq, value, label='Spectrum')
plt.plot(f_max, value[i_max], 'o', label='Max Frequency = 90 bpm')
plt.xlabel('Frequency (bpm)')
plt.grid()
plt.legend()

In [ ]:
# Peak-to-Peak Interval Interpolation
peak_time = peaks / 256
t = peak_time[1:]
d = peak_time[1:] - peak_time[:-1]
t_interp = np.linspace(t.min(), t.max(), 1000)
s_interp = interpolate.interp1d(t, d, kind='cubic')(t_interp)

print(len(d))
plt.plot(t, d)
plt.plot(t_interp, s_interp)
plt.xlabel('Time (s)')
plt.ylabel('Interavl (s)')

In [ ]:
# HRV calculation
s_fft = np.fft.fft(s_interp - s_interp.mean())
s_freq = np.fft.fftfreq(len(s_interp), d=t_interp[1] - t_interp[0])

f_sample = 1 / (t_interp[1] - t_interp[0])
s_psd = (1/(f_sample*len(s_interp))) * abs(s_fft) ** 2

f_step = s_freq[1]
r_vlf = (np.array([0.0033, 0.04]) / f_step + 0.5).astype(int)
r_hf = (np.array([0.15, 0.4]) / f_step + 0.5).astype(int)
r_lf = (np.array([0.04, 0.15]) / f_step + 0.5).astype(int)

range_vlf = range(r_vlf[0], r_vlf[1]+1)
range_hf = range(r_hf[0], r_hf[1]+1)
range_lf = range(r_lf[0], r_lf[1]+1)

VLF_power = s_psd[range_vlf].sum() * f_step * 1000000
LF_power = s_psd[range_lf].sum() * f_step * 1000000
HF_power = s_psd[range_hf].sum() * f_step * 1000000
LF_HF = LF_power / HF_power

LF_peak = s_freq[range_lf][np.argmax(s_psd[range_lf])]
HF_peak = s_freq[range_hf][np.argmax(s_psd[range_hf])]

SDNN = d.std()

plt.plot(s_freq[range_vlf], s_psd[range_vlf], label='VLF')
plt.plot(s_freq[range_hf], s_psd[range_hf], label='HF')
plt.plot(s_freq[range_lf], s_psd[range_lf], label='LF')
plt.xlim([0, 0.4])
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (sec^2 / Hz)')
plt.legend()

print('HRV')
print()
#print('SDNN      : %.3f ms' % SDNN)
print('VLF power : %8.3f ms^2' % VLF_power)
print('LF power  : %8.3f ms^2' % LF_power)
print('HF power  : %8.3f ms^2' % HF_power)
print('LF peak   : %8.3f Hz' % LF_peak)
print('HF peak   : %8.3f Hz' % HF_peak)
print('LF/HF     : %8.3f' % LF_HF)

print()
